In [1]:
import numpy as np 
import pandas as pd 
import os 
import sys 
import plotly.graph_objects as go 
import re

import pyKVFinder

In [6]:
OR_LEARNING_PATH = os.path.join(os.getcwd().split('OR_learning')[0], 'OR_learning/')
sys.path.insert(0, os.path.join(OR_LEARNING_PATH, 'utils/'))

import voxel_functions as vf
import color_function as cf 
import plot_functions as pf 
import BindingCavity_functions as bc 

In [7]:
import importlib 

importlib.reload(bc)
importlib.reload(vf)
importlib.reload(pf)

<module 'plot_functions' from '/mnt/data2/Justice/OR_learning/utils/plot_functions.py'>

### Defining Canonical binding cavity
This notebook aims to conduct similar comparison as bc_0_binding_cavity.ipynb. However, isntead of taking the whole structure / cavity. Here, we aim to segregate and identify the main binding pocket and comparing mainly the upper half of ORs.


### Loading data 

In [ ]:
# Read pickle file for data 
bc_cav_coords = pd.read_pickle('/data/jlu/OR_learning/files/dict_bc_pyKVcav_tmaligned.pkl')
bc_res_coords = pd.read_pickle('/data/jlu/OR_learning/files/dict_bc_pyKVres_tmaligned.pkl')

# Create cavity voxels from coordinates 
# DROP Or defined in the exclusion list below
EXCLUDE_OR_LIST = ['Or4Q3', 'Or2W25', 'Or2l1', 'Or4A67', 'Or2I1']
bc_cav_coords = {key: value for key, value in bc_cav_coords.items() if key not in EXCLUDE_OR_LIST}
bc_res_coords = {key: value for key, value in bc_res_coords.items() if key not in EXCLUDE_OR_LIST}

# DROP non DL_OR names
bc_cav_coords = {key: value for key, value in bc_cav_coords.items() if key.startswith('Or')}
bc_res_coords = {key: value for key, value in bc_res_coords.items() if key.startswith('Or')}

voxelized_cav_res, voxel_shape = vf.voxelize_cavity(list(bc_cav_coords.values()), 
                                                     list(bc_res_coords.values()), resolution=1.5)

# Output: List of 1D arrays representing voxelized space
print(np.array(voxelized_cav_res).shape)
# pd.to_pickle(voxelized_cav_res, '../../output/bc_voxel/voxelized_cav_res.pkl')

In [ ]:
# voxelized_cav_res = pd.read_pickle('../../output/bc_voxel/voxelized_cav_res.pkl')

In [ ]:
Olfr_DL = pd.read_csv('../../files/Olfr_DL.csv', index_col=0)
# Olfr_DL.loc[Olfr_DL.Olfr.isin(['Olfr1377', 'Olfr224', 'Olfr330'])]
# Olfr_DL.loc[Olfr_DL.DL_OR.isin(['Or51E2', 'Or51E1'])]

### Defining upper bounds

- Read in aa alignment via `/files/mouseOR_60p_aaIdentity.fasta`
- Manually define the upper half of ORs coordinates 
- Using the alignment only read in residues aligned with the location from pdb 


#### Script to generate cavity coordinates 

In [ ]:
"""
Updated bc_cav_dump.py script to also output surface coordinates -> bc_cavsurf_coord

DO NOT RUN HERE. files generated via bc_cav_dump.py script. 

"""
# AF2_PATH = '/data/jlu/AF_files/AF_tmaligned_pdb'
# pdb_files = os.listdir(AF2_PATH)
# Olfr_DL = pd.read_csv('/data/jlu/OR_learning/files/Olfr_DL.csv', index_col=0)

# Olfr_DL = dict(zip(Olfr_DL.Olfr, Olfr_DL.DL_OR))

# TEST_OR_LIST = ['Or51e2', 'Olfr1377']
# pdb_files = [_pdb for _pdb in pdb_files for _OR in TEST_OR_LIST if _OR in _pdb]

# # Running pyKVFinder standard workflow for cavity grid
# bc_cav_coords = {}
# bc_cavsurf_coords = {}
# bc_res_coords = {}

# for _pdb in pdb_files: 
#     _olfr = _pdb.split('_')[0]
#     # if (_olfr in list(Olfr_DL.Olfr) or (_olfr in list(Olfr_DL.DL_OR))): 
#     bc_results = pyKVFinder.run_workflow(os.path.join(AF2_PATH, _pdb))
#     bc_atomic = pyKVFinder.read_pdb(os.path.join(AF2_PATH, _pdb))
    
#     # Store cavity coordinates in dict with DL_OR name
#             # Store cavity coordinates in dict with DL_OR name
#     combined_coords = []
#     bc_results_coord = bc.grid2coords(bc_results)
#     # Append cavity coordinates into dict 
#     for _cav_coords in bc_results_coord[0].values(): 
#         combined_coords.extend(_cav_coords)
#     bc_cav_coords[Olfr_DL.get(_olfr, _olfr)] = combined_coords
    
#     combined_coords = []
#     # Append cavity surface coordinates into dict 
#     for _cavsurf_coords in bc_results_coord[1].values(): 
#         combined_coords.extend(_cavsurf_coords)
#     bc_cavsurf_coords[Olfr_DL.get(_olfr, _olfr)] = combined_coords
    
    
#     # Get cavity interacting residue coords 
#     res_coords = []
#     res_coords_dict = bc.res2atomic(bc_results, bc_atomic)
#     for _res in res_coords_dict: 
#         res_coords.extend(res_coords_dict[_res][:, [0,2,3,4,5,6]].tolist())  # Extract [ResNum, AA, Atom, x, y, z]
#     # remove duplicated residues 
#     bc_res_coords[Olfr_DL.get(_olfr, _olfr)] = [list(x) for x in set(tuple(entry) for entry in res_coords)]    

#### Filter for **Canonical Binding Cavity** coordinates 
**Canonical Binding Cavity (Cbc)**
<br>Cbc is defined by overlaying Or1A1, Or51E2 ... etc predefined receptors
<br>plus ~30 receptors that spans across the Odor Response metric to capture diverse amount of ORs. 
<br>After superimposing the OR's binding cavity together, the cavity space intersecting at least 10% of cavity space is defined as Cbc.
<br><br>For rationale visit `Optimizing Cbc definition . . .` below  

In [18]:
# Read pickle file for data 
bc_cav_coords = pd.read_pickle(os.path.join(OR_LEARNING_PATH, 'files/binding_cavity/dict_bc_pyKVcav_tmaligned.pkl'))
bc_cavsurf_coords = pd.read_pickle(os.path.join(OR_LEARNING_PATH, 'files/binding_cavity/dict_bc_pyKVcavsurf_tmaligned.pkl'))
bc_res_coords = pd.read_pickle(os.path.join(OR_LEARNING_PATH, 'files/binding_cavity/dict_bc_pyKVres_tmaligned.pkl'))

# Create cavity voxels from coordinates 
# DROP Or defined in the exclusion list below
EXCLUDE_OR_LIST = ['Or4Q3', 'Or2W25', 'Or2l1', 'Or4A67', 'Or2I1']
bc_cav_coords = {key: value for key, value in bc_cav_coords.items() if key not in EXCLUDE_OR_LIST}
bc_res_coords = {key: value for key, value in bc_res_coords.items() if key not in EXCLUDE_OR_LIST}

Olfr_DL = pd.read_csv(os.path.join(OR_LEARNING_PATH, 'files/Olfr_DL.csv'), index_col=0)
Olfr_DL = dict(zip(Olfr_DL.Olfr, Olfr_DL.DL_OR))

# Translate to DL_OR
bc_cav_coords = {Olfr_DL.get(_olfr, _olfr): bc_cav_coords[_olfr] for _olfr in bc_cav_coords.keys()}
bc_res_coords = {Olfr_DL.get(_olfr, _olfr): bc_res_coords[_olfr] for _olfr in bc_res_coords.keys()}
bc_cavsurf_coords = {Olfr_DL.get(_olfr, _olfr): bc_cavsurf_coords[_olfr] for _olfr in bc_cavsurf_coords.keys()}

# DROP non DL_OR names
bc_cav_coords = {key: value for key, value in bc_cav_coords.items() if key.startswith('Or')}
bc_res_coords = {key: value for key, value in bc_res_coords.items() if key.startswith('Or')}
bc_cavsurf_coords = {key: value for key, value in bc_cavsurf_coords.items() if key.startswith('Or')}



In [356]:
"""
Defines the overall binding cavity zone. . . 

Define the ORs used for identifying the zone. 
Biggest binding cavity in each ORs is obtained and expanded x amount. 

The zone is then super imposed on each other. 

DO NOT RUN HERE. . . 
"""

# from sklearn.cluster import DBSCAN

# odorResp = pd.read_csv('../../files/Deorphanization/compiled_odor_sigResp_wide.csv', index_col=0)
# odorResp.columns = [Olfr_DL.get(_col, _col) for _col in odorResp] # Change to DL_OR

# DEFINE_CAVZONE_OR = ['Or51e2', 'Or1Ad1', 'Or55B4'] + list(odorResp.sum().sort_values().index[::10])
# DEFINE_CAVZONE_OR = [_Or for _Or in DEFINE_CAVZONE_OR if _Or in list(bc_cavsurf_coords.keys())]

# # Getting expanded coords for defined ORs 
# expanded_coords, largest_cavity_coords = bc.define_binding_cavity_zone({_Or : bc_cavsurf_coords[_Or] for _Or in DEFINE_CAVZONE_OR}, 
#                                                                        expansion_distance=3)

# # Trim coordinates to speed up procees in making voxel 
# expanded_coords_trimmed = {}
# for _Or in expanded_coords: 
#     expanded_coords_trimmed[_Or] = np.unique(expanded_coords[_Or] // 1, axis=0)
    
# # Identify Canonical Binding Cavity by threshold intersection of expanded cavity coordinates 
# threshold = 0.1
# min_count = int(threshold * len(expanded_coords_trimmerd))
# all_coordinates = np.vstack(list(expanded_coords_trimmerd.values()))
# unique_coordinates, counts = np.unique(all_coordinates, axis=0, return_counts=True)

# canonical_bc_coords = unique_coordinates[counts >= min_count]


# # Filtering ORs cavity and residue coordinates if they overlap with the defined canonical binding cavity
# Cbc_cav_coords_ = { _Or: bc.filter_coordinates_within_cavity(canonical_bc_coords, 
#                                                              np.array(bc_cavsurf_coords[_Or])) for _Or in bc_cavsurf_coords}

# Cbc_res_coords = { _Or: bc.filter_coordinates_within_cavity(canonical_bc_coords, 
#                                                              np.array(bc_res_coords[_Or]), 
#                                                              is_residue=True) for _Or in bc_res_coords}
# import pickle 
# with open('/data/jlu/OR_learning/files/dict_Cbc_cav_coords.pkl', 'wb') as f:
#     pickle.dump(Cbc_cav_coords_, f)
# with open('/data/jlu/OR_learning/files/dict_Cbc_res_coords.pkl', 'wb') as f:
#     pickle.dump(Cbc_res_coords, f)

In [3]:
# Read in canonical binding cavity filtered coordinates 
Cbc_cav_coords = pd.read_pickle(os.path.join(OR_LEARNING_PATH,'files/binding_cavity/dict_Cbc_cav_coords.pkl'))
Cbc_res_coords = pd.read_pickle(os.path.join(OR_LEARNING_PATH,'files/binding_cavity/dict_Cbc_res_coords.pkl'))
canonical_bc_coords = pd.read_pickle(os.path.join(OR_LEARNING_PATH,'files/binding_cavity/canonical_bc_coords.pkl'))

# DROP non DL_OR names
Cbc_cav_coords = {key: value for key, value in Cbc_cav_coords.items() if key.startswith('Or')}
Cbc_res_coords = {key: value for key, value in Cbc_res_coords.items() if key.startswith('Or')}

In [ ]:


TEST_OR_LIST = ['Or1Ad1', 'Or51e2'] + list(Cbc_cav_coords.keys())[::300] 

TEST_cav_coords = {_olfr: Cbc_cav_coords[_olfr] for _olfr in TEST_OR_LIST if _olfr in Cbc_cav_coords.keys()}
TEST_res_coords = {_olfr: Cbc_res_coords[_olfr] for _olfr in TEST_OR_LIST if _olfr in Cbc_res_coords.keys()}

TEST_cav_coords['CAVITY_ZONE'] = list(canonical_bc_coords) 
                                                    
_pdb = bc.load_pdb_coordinates(os.path.join('../../../AF_files/AF_tmaligned_pdb/Or51e2_Mol2.3_Olfr78_Psgr_tmaligned.pdb')) # For backbone visualization
TEST_cav_coords['BACKBONE'] =list(_pdb[1]) # Adding cavity zone together

TEST_voxelized_cavities, TEST_voxel_shape = vf.voxelize_cavity(list(TEST_cav_coords.values()), 
                                                               list(TEST_res_coords.values()), 
                                                               resolution=1)


voxel_data = TEST_voxelized_cavities
color_map = cf.distinct_colors(list(TEST_cav_coords.keys()))

color_map['BACKBONE'] = '#D3D3D3'
color_map['CAVITY_ZONE'] = 'black'

fig = pf.visualize_voxel_grid(voxel_data, 
                              coordinate_labels=list(TEST_cav_coords.keys()), 
                              color_map=color_map, 
                              opacity=0.03,
                              highlight_labels=TEST_OR_LIST + ['BACKBONE'])
fig.show()
# fig.write_html('../../output/Canonical_bc/Cbc_cavres.html')

#### Optimizing and checking for **Cbc** definition

In [ ]:
"""
Define cavity zone relative to binding cavity detected via pyKVfinder in ORs
"""

import plotly.graph_objects as go

TEST_OR_LIST = DEFINE_CAVZONE_OR

# To visualize largest cavity
TEST_cavsurf_coords = {_olfr: largest_cavity_coords[_olfr] for _olfr in TEST_OR_LIST if _olfr in largest_cavity_coords.keys()}
# To visualize all binding cavity
# TEST_cavsurf_coords = {_olfr: bc_cavsurf_coords[_olfr] for _olfr in TEST_OR_LIST if _olfr in bc_cavsurf_coords.keys()} 
TEST_cavsurf_coords['CAVITY_ZONE'] = np.concatenate(list(expanded_coords.values())) # Adding cavity zone together
                                                    
_pdb = bc.load_pdb_coordinates(os.path.join('/data/jlu/AF_files/AF_tmaligned_pdb/Olfr1377_tmaligned.pdb')) # For backbone visualization
TEST_cavsurf_coords['BACKBONE'] =list(_pdb[1]) # Adding cavity zone together

TEST_voxelized_cavities, TEST_voxel_shape = vf.voxelize_cavity(list(TEST_cavsurf_coords.values()), 
                                                               resolution=1)


voxel_data = TEST_voxelized_cavities
color_map = cf.distinct_colors(list(TEST_cavsurf_coords.keys()))
color_map['BACKBONE'] = '#D3D3D3'
color_map['CAVITY_ZONE'] = '#D3D3D3'


fig = pf.visualize_voxel_grid(voxel_data, 
                              coordinate_labels=list(TEST_cavsurf_coords.keys()), 
                              color_map=color_map, 
                              opacity=0.01,
                              highlight_labels=TEST_OR_LIST + ['BACKBONE'])
fig.show()

# fig.write_html('../../output/Canonical_bc/Define_bc/DEFINE_bc_exp3_allcav.html')
# fig.write_html('../../output/Canonical_bc/Define_bc/DEFINE_bc_exp3.html')

In [ ]:
"""
Intersected binding cavity zone is used instead as the canonical binding cavity. 
"""

# Get unique coordinates to speed up procees in making voxel 
unique_coords = {}
for _Or in expanded_coords: 
    unique_coords[_Or] = np.unique(expanded_coords[_Or] // 1, axis=0)
    
# Threshold for intersection 
threshold = 0.1
min_count = int(threshold * len(unique_coords))
all_coordinates = np.vstack(list(unique_coords.values()))
unique_coordinates, counts = np.unique(all_coordinates, axis=0, return_counts=True)
intersected_coordinates = unique_coordinates[counts >= min_count]

TEST_new_coords = {}
TEST_new_coords['CAVITY_ZONE'] = np.unique(all_coordinates, axis=0)
TEST_new_coords['INTERSECTED_CAVITY_ZONE'] = list(intersected_coordinates)

_pdb = bc.load_pdb_coordinates(os.path.join('/data/jlu/AF_files/AF_tmaligned_pdb/Olfr1377_tmaligned.pdb')) # For backbone visualization
TEST_new_coords['BACKBONE'] =list(_pdb[1]) # Adding cavity zone together

TEST_voxelized_cavities, TEST_voxel_shape = vf.voxelize_cavity(list(TEST_new_coords.values()), 
                                                               resolution=1)

voxel_data = TEST_voxelized_cavities
color_map = cf.distinct_colors(list(TEST_new_coords.keys()))
color_map['BACKBONE'] = '#D3D3D3'

fig = pf.visualize_voxel_grid(voxel_data, 
                              coordinate_labels=list(TEST_new_coords.keys()), 
                              color_map=color_map, 
                              opacity=0.01,
                              highlight_labels=['BACKBONE'])
fig.show()

# fig.write_html('../../output/Canonical_bc/Define_bc/DEFINE_bc_intersectcav10p.html')

In [ ]:
"""
Plotting cavity surface filtered via intersected coordinates cavity zone. 

"""

TEST_OR_LIST = DEFINE_CAVZONE_OR

TEST_new_coords = { _Or: bc.filter_coordinates_within_cavity(intersected_coordinates, 
                                                             np.array(bc_cavsurf_coords[_Or])) for _Or in TEST_OR_LIST}
TEST_new_coords['INTERSECTED_CAVITY_ZONE'] = list(intersected_coordinates)

_pdb = bc.load_pdb_coordinates(os.path.join('/data/jlu/AF_files/AF_tmaligned_pdb/Olfr1377_tmaligned.pdb')) # For backbone visualization
TEST_new_coords['BACKBONE'] =list(_pdb[1]) # Adding cavity zone together

TEST_voxelized_cavities, TEST_voxel_shape = vf.voxelize_cavity(list(TEST_new_coords.values()), 
                                                               resolution=1)

voxel_data = TEST_voxelized_cavities
color_map = cf.distinct_colors(list(TEST_new_coords.keys()))
color_map['INTERSECTED_CAVITY_ZONE'] = '#D3D3D3'
color_map['BACKBONE'] = '#D3D3D3'

fig = pf.visualize_voxel_grid(voxel_data, 
                              coordinate_labels=list(TEST_new_coords.keys()), 
                              color_map=color_map, 
                              opacity=0.02,
                              highlight_labels=TEST_OR_LIST + ['BACKBONE'])
fig.show()

# fig.write_html('../../output/Canonical_bc/Define_bc/DEFINE_bc_intersectcav10p_cavsurf.html')

In [ ]:
"""
Plotting cavity residues filtered via intersected coordinates cavity zone. 

"""

TEST_OR_LIST = DEFINE_CAVZONE_OR[0:1]

TEST_res_coords = { _Or: bc.filter_coordinates_within_cavity(intersected_coordinates, 
                                                             np.array(bc_res_coords[_Or]), 
                                                             is_residue=True) for _Or in TEST_OR_LIST}

TEST_new_coords = { _Or: bc.filter_coordinates_within_cavity(intersected_coordinates, 
                                                             np.array(bc_cavsurf_coords[_Or])) for _Or in TEST_OR_LIST}
TEST_new_coords['INTERSECTED_CAVITY_ZONE'] = list(intersected_coordinates)

_pdb = bc.load_pdb_coordinates(os.path.join('/data/jlu/AF_files/AF_tmaligned_pdb/Olfr1377_tmaligned.pdb')) # For backbone visualization
TEST_new_coords['BACKBONE'] =list(_pdb[1]) # Adding cavity zone together

TEST_voxelized_cavities, TEST_voxel_shape = vf.voxelize_cavity(list(TEST_new_coords.values()), 
                                                               list(TEST_res_coords.values()),
                                                               resolution=1)

voxel_data = TEST_voxelized_cavities
color_map = cf.distinct_colors(list(TEST_new_coords.keys()))
color_map['INTERSECTED_CAVITY_ZONE'] = '#D3D3D3'
color_map['BACKBONE'] = '#D3D3D3'

fig = pf.visualize_voxel_grid(voxel_data, 
                              coordinate_labels=list(TEST_new_coords.keys()), 
                              color_map=color_map, 
                              opacity=0.05,
                              highlight_labels=TEST_OR_LIST + ['BACKBONE'])
fig.show()

# fig.write_html('../../output/binding_cavity_upper/Define_bc/DEFINE_bc_intersectcav10p_cavres.html')

### Residue property visualization

In [ ]:
# TEST visualize binding cavity alignment in voxel form
"""
Testing workflow of 
- binding cavity coordinates > voxelized coordinate 
visualization to ensure the alignment is preserved 
"""

TEST_OR_LIST = ['Or1Ad1']

TEST_cav_coords = {_olfr: Cbc_cav_coords[_olfr] for _olfr in TEST_OR_LIST if _olfr in Cbc_cav_coords.keys()}
TEST_res_coords = {_olfr: Cbc_res_coords[_olfr] for _olfr in TEST_OR_LIST if _olfr in Cbc_res_coords.keys()}

TEST_voxelized_cavities, TEST_voxel_shape = vf.voxelize_cavity(list(TEST_cav_coords.values()), 
                                                               list(TEST_res_coords.values()), resolution=1)

# Output: List of 1D arrays representing voxelized space
print(np.array(TEST_voxelized_cavities).shape)
voxel_data = TEST_voxelized_cavities
voxel_size = 1

class_names = [f"{i}" for i in range(8)]
color_map = cf.distinct_colors(list(range(len(class_names))))

# Create a plotly scatter plot
fig = go.Figure()

# Loop through each cavity
for i, voxel_grid in enumerate(voxel_data):
    # Loop through each one-hot encoded class (index in the last dimension)
    for class_index in range(8):  # Adjust if the one-hot length changes
        # Find the indices of the voxels with a 1 in the current class_index
        occupied_voxels = np.array(np.where(voxel_grid[..., class_index] == 1)).T
        
        # Skip if no voxels are occupied for the current class
        if len(occupied_voxels) == 0:
            continue
        
        # Convert voxel indices back to 3D space coordinates
        x = occupied_voxels[:, 0] * voxel_size
        y = occupied_voxels[:, 1] * voxel_size
        z = occupied_voxels[:, 2] * voxel_size
        
        # Add the points for the current class to the plot
        fig.add_trace(go.Scatter3d(
            x=x, y=y, z=z,
            mode='markers',
            name=f"{TEST_OR_LIST[i]} - {class_names[class_index]}",
            marker=dict(
                size=3,
                color=color_map[class_index] if class_index != 0 else '#D3D3D3',
                opacity=0.5
            )
        ))

# Update layout for 3D visualization
fig = pf._plotly_blank_style(fig)

fig.show()
# fig.write_html('../../output/Canonical_bc/misc/TEST_bc_voxel_res_property.html')

### . . . 